In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
!ls

 Complete.pdf				  PDF_Folder_Biomedical
'covid_question_answer_dataset (1).csv'   PDF_Folder_Financial
 covid_question_answer_dataset.csv	  PDF_Folder_Scientific
 CrossDomainAnalysis.ipynb		 ' QA_system_for_BioMedical_data.ipynb'
 DistilBERT.ipynb			  QA_system_for_scientific_data.ipynb
 EvaluationScript_financial.ipynb	  qa-with-squad-2.0-dataset-and-bert-base.ipynb
 FinBERT.ipynb				  question_answer_dataset_financial.csv
'FineTuningDistillBert_SQAUD .ipynb'	  scientific_question_answer_dataset.csv
 FineTuningDistillBert_SQAUD_.ipynb	  Untitled0.ipynb
 Make_dataset_from_PDFs.ipynb		  Untitled1.ipynb
 my_awesome_qa_model


In [ ]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("allenai/scibert_scivocab_uncased")

In [ ]:
from transformers import AutoTokenizer, AutoModel
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/scientific_question_answer_dataset.csv")

In [ ]:
df

,Question,Context,GroundTruth
0,What does this paper present?,\nThis paper presents an overview of a quality...,This paper presents an overview of a quality s...
1,What types of DL models are used in the qualit...,\nThis paper presents an overview of a quality...,"Two types of DL models, a classification and e..."
2,How are abstracts classified in this system?,\nThis paper presents an overview of a quality...,The abstracts of the scientific literature are...
3,What types of information are extracted by the...,\nThis paper presents an overview of a quality...,The question and answering model extracts info...
4,Which model is used as the baseline for classi...,\nThis paper presents an overview of a quality...,The Bidirectional Encoder Representations of T...
5,How many EMF-related research papers are used ...,\nThis paper presents an overview of a quality...,The models are fine-tuned with 455 EMF-related...
6,What improvements were observed in the fine-tu...,\nThis paper presents an overview of a quality...,The fine-tuned model showed improved performan...
7,What is the ultimate goal of the study?,\nThis paper presents an overview of a quality...,The ultimate goal of the study is to develop a...
8,How does the software system categorize EMF-re...,\nThis paper presents an overview of a quality...,The software system processes EMF-related scie...
9,What are the different evaluation strategies m...,\nThis paper presents an overview of a quality...,Different evaluation strategies are required f...


In [ ]:
from sklearn.metrics import f1_score
import re

def answer_question4(question, answer_text, ground_truth):
    ground_truth_processed = re.sub(r' ##', '', ground_truth)
    # Tokenize and preprocess the ground truth
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)
    # Ensure that ground truth and predicted answer have the same length
    max_length = max(len(ground_truth_processed), len(predicted_answer_processed))
    ground_truth_processed = ground_truth_processed.ljust(max_length)
    predicted_answer_processed = predicted_answer_processed.ljust(max_length)
    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted', zero_division=1)
    return predicted_answer, f1_ground_truth_predicted

In [ ]:
# Loop through rows and call the function
for index, row in df.iterrows():
    user_question = row['Question']
    context = row['Context']
    ground_truth = row['GroundTruth']

    # Call your answer_question function
    ans, f1 = answer_question4(user_question,context,ground_truth)
    # Print the result
    print(f"Question: {user_question}")
    print(f"Context: {context}")
    print(f"Answer: {ans}")
    print(f"F1 Score: {f1}")
    print("=" * 50)

The input has a total of 134 tokens.
Question: What does this paper present?
Context: 
This paper presents an overview of a quality scoring system that
utilizes pre-trained deep neural network models. Two types of DL
models, a classification and extractive question answering (EQA)
models are used to implement components of the system.
.
.
.
 The number of animals can be an important factor in an in-vivo study; however, the same criteria are not applicable for an in-vitro study. Copyright is held by the authors.
APPLIED COMPUTING REVIEW MAR. 2022, VOL. 22, NO. 1 31
Figure 1: Overview of a paper quality scoring system
Figure 1 shows an overview of the software system. T

Answer: held by the authors . applied computing review mar . 2022 , vol . 22 ,
F1 Score: 0.05017406787318292
The input has a total of 141 tokens.
Question: What types of DL models are used in the quality scoring system?
Context: 
This paper presents an overview of a quality scoring system that
utilizes pre-trained deep n